# Diffusion coefficient from a VASP file

[Previously](./vasp_msd.html), we looked at obtaining accurate estimates for the mean-squared displacement with `kinisi`. 
Here, we show using the same class to evaluate the diffusion coefficient, using the `kinisi` [methodology](./methodology.html).

In [ ]:
import numpy as np
from kinisi.analyze import DiffusionAnalyzer
from pymatgen.io.vasp import Xdatcar
np.random.seed(42)

There the `p_params` dictionary describes details about the simulation, and are documented in the [parser module](./parser.html) (also see the [MSD Notebook](./vasp_msd.html)).

In [ ]:
p_params = {'specie': 'Li',
            'time_step': 2.0,
            'step_skip': 50}

While the `b_params` dictionary describes the details of the bootstrapping process, these are documented in the [diffusion module](./diffusion.html#kinisi.diffusion.MSDBootstrap). Here, we indicate that we only want to investigate diffusion in the *xy*-plane of the system.

In [ ]:
b_params = {'dimension': 'xy'}

In [ ]:
xd = Xdatcar('./example_XDATCAR.gz')
diff = DiffusionAnalyzer.from_Xdatcar(xd, parser_params=p_params, bootstrap_params=b_params)

In the above cells, we parse and determine the uncertainty on the mean-squared displacement as a function of the timestep. 
We should visualise this, to check that we are observing diffusion in our material and to determine the timescale at which this diffusion begins. 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.errorbar(diff.dt, diff.msd, diff.msd_std)
plt.axvline(diff.ngp_max, color='g')
plt.ylabel('MSD/Å$^2$')
plt.xlabel('$\Delta t$/ps')
plt.show()

The vertical green line indicates the start of the diffusive regime, based on the maximum of the [non-Gaussian parameter](https://doi.org/10.1073/pnas.1900239116). 
We can also visualise this on a log-log scale, which helps to reveal the diffusive regime. 

In [ ]:
plt.errorbar(diff.dt, diff.msd, diff.msd_std)
plt.axvline(diff.ngp_max, color='g')
plt.ylabel('MSD/Å$^2$')
plt.xlabel('$\Delta t$/ps')
plt.yscale('log')
plt.xscale('log')
plt.show()

It appears that the non-Gaussian parameter does okay at predicting the of the diffusive regime for this example (it is slightly underestimated), therefore we will set the start of the diffusive regime as the `ngp_max` object. 
If the non-Gaussian parameter gave an incorrect value, the start of the diffusive regim could be set to, say, 4 ps using the keyword argument of `start_dt=4`. 
We can then run the diffusion analysis as follows. 

In [ ]:
diff.diffusion(diff.ngp_max)

This method estimates the correlation matrix between the timesteps and uses likelihood sampling to find the diffusion coefficient, $D$ and intercept (the units are cm\ :sup:`2`\ /s and Å\ :sup:`2`  respectively).
Now we can probe these objects.

We can get the median and 95 % confidence interval using, 

In [ ]:
diff.D.n, diff.D.ci()

In [ ]:
diff.intercept.n, diff.intercept.ci()

Or we can get all of the sampled values from one of these objects. 

In [ ]:
diff.D.samples

Having completed the analysis, we can save the object for use later (such as downstream by a plotting script). 

These hidden cells exist to remove any existing `example.hdf` file on builiding.

In [ ]:
!rm example.hdf

In [ ]:
diff.save('example.hdf')

The data is saved in a HDF5 file format which helps us efficiently store our data. 
We can then load the data with the following class method. 

In [ ]:
loaded_diff = DiffusionAnalyzer.load('example.hdf')

We can plot the data with the credible intervals from the $D$ and $D_{\text{offset}}$ distribution.

In [ ]:
credible_intervals = [[16, 84], [2.5, 97.5], [0.15, 99.85]]
alpha = [0.6, 0.4, 0.2]

plt.plot(loaded_diff.dt, loaded_diff.msd, 'k-')
for i, ci in enumerate(credible_intervals):
    plt.fill_between(loaded_diff.dt,
                     *np.percentile(loaded_diff.distribution, ci, axis=1),
                     alpha=alpha[i],
                     color='#0173B2',
                     lw=0)
plt.ylabel('MSD/Å$^2$')
plt.xlabel('$\Delta t$/ps')
plt.show()

Additionally, we can visualise the distribution of the diffusion coefficient that has been determined.

In [ ]:
plt.hist(loaded_diff.D.samples, density=True)
plt.axvline(loaded_diff.D.n, c='k')
plt.xlabel('$D$/cm$^2$s$^{-1}$')
plt.ylabel('$p(D$/cm$^2$s$^{-1})$')
plt.show()

Or the joint probability distribution for the diffusion coefficient and intercept.

In [ ]:
from corner import corner

In [ ]:
corner(loaded_diff.flatchain, labels=['$D$/cm$^2$s$^{-1}$', 'intercept/Å$^2$'])
plt.show()

In [ ]:
ppd = loaded_diff.posterior_predictive({'n_posterior_samples': 128,
                                        'progress': False})

In [ ]:
plt.plot(loaded_diff.dt, loaded_diff.msd, 'k-')
for i, ci in enumerate(credible_intervals):
    plt.fill_between(loaded_diff.dt[5:],
                     *np.percentile(ppd.T, ci, axis=1),
                     alpha=alpha[i],
                     color='#0173B2',
                     lw=0)
plt.ylabel('MSD/Å$^2$')
plt.xlabel('$\Delta t$/ps')
plt.show()